In [107]:
import re
import json
import urllib
import urllib2
import StringIO
import datetime
import pandas as pd
import os
import numpy as np
import plotly
pd.set_option('display.max_colwidth', 5000)

HOST = 'http://mtlog01-01-1.yandex.ru:8123/'
import requests
def get_clickhouse_data(query, host = HOST, connection_timeout = 1500):
    r = requests.post(host, params = {'query': query}, timeout = connection_timeout)
    if r.status_code == 200:
        return r.text
    else:
        raise ValueError, r.text
        
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(StringIO.StringIO(data), sep = '\t')
    return df

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import requests
import StringIO
import pandas as pd

print __version__ # need 1.9.0 or greater

init_notebook_mode(connected = True)

def plotly_df_preliminary(df, title = ''):
    data = []
    for column in df.columns:
        if column != datetime.datetime.today().strftime('%Y-%m-%d'):
            trace = go.Scatter(
                x = df.index,
                y = df[column],
                mode = 'lines',
                name = column
            )
            data.append(trace)
        else:
            
            tmp_df = df.loc[:limit_time]
            trace = go.Scatter(
                x = tmp_df[:-3].index,
                y = tmp_df[column][:-3],
                mode = 'lines',
                name = column
            )
            data.append(trace)
            
            trace = go.Scatter(
                x = tmp_df[-4:].index,
                y = tmp_df[column][-4:],
                mode = 'lines',
                name = column + ' not complete',
                line = dict(dash = 'dashdot', color = ('#2fa916'))
            )
            data.append(trace)
        
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    # iplot(fig, show_link=False)
    
    html = plotly.offline.plot(fig, show_link=False, output_type='div')
    return html

def plotly_df(df, title = ''):
    data = []
    
    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)
    
    html = plotly.offline.plot(fig, show_link=False, output_type='div')
    return html


1.12.9


In [121]:
q = '''
    SELECT
        sum(Sign) as total_visits,
        sumIf(Sign, SearchEngineID IN (2, 13, 181, 225)) as ya_visits,
        sumIf(Sign, SearchEngineID IN (6, 119)) as mail_visits,
        sumIf(Sign, SearchEngineID IN (3, 68, 224)) as google_visits,
        toStartOfFiveMinute(StartTime) as time,
        StartDate as date
    FROM visits_all
    WHERE StartDate IN (today(), today() - 1)
        AND SearchEngineID != 0 AND NOT IsYandex
        AND NOT IsRobot
        AND regionToCountry(RegionID) = 225
    GROUP BY date, time
    FORMAT TabSeparatedWithNames
'''

df = get_clickhouse_df(q)

In [122]:
df['time'] = map(lambda x: x[-8:-3], df.time)

In [125]:
for param in ['mail', 'google', 'ya']:
    df[param + '_share'] = 100.*df[param + "_visits"]/df.total_visits
    tmp_df = df.pivot_table(index = 'time', columns = 'date', values = param + '_visits')
    tmp_df['2017-05-23']['15:20': ] = np.nan
    plotly_df(tmp_df, title = param)

In [124]:
q = '''
    SELECT SearchEngineID, Referer, UTCStartTime, TraficSourceID
    FROM visits_all
    WHERE StartDate = today() AND Referer LIKE '%news-cache/touch%'
        AND UTCStartTime >= '2017-05-23 15:12:00'
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''
get_clickhouse_df(q)

,SearchEngineID,Referer,UTCStartTime,TraficSourceID
0,0,https://yandex.ru/search/news-cache/touch?text=https://rg.ru/2017/05/23/megafon-rasskazal-kak-poluchit-kompensaciiu-posle-sboia.html,2017-05-23 15:21:36,3
1,0,https://yandex.ru/search/news-cache/touch?text=http://sarnovosti.ru/news.php?ID=69177,2017-05-23 15:14:36,1
2,0,https://yandex.ru/search/news-cache/touch?text=http://sarnovosti.ru/news.php?ID=69220,2017-05-23 15:16:12,1
3,0,https://yandex.ru/search/news-cache/touch?text=https://rg.ru/2017/05/23/megafon-rasskazal-kak-poluchit-kompensaciiu-posle-sboia.html,2017-05-23 15:13:24,1
4,0,https://yandex.ru/search/news-cache/touch?text=https://rg.ru/2017/05/23/megafon-rasskazal-kak-poluchit-kompensaciiu-posle-sboia.html,2017-05-23 15:13:55,3


In [112]:
q = '''
    SELECT SearchEngineID, Referer, TraficSourceID, UTCStartTime
    FROM visits_all
    WHERE StartDate = today() AND Referer LIKE '%go.mail.ru/redir%'
        AND UTCStartTime >= '2017-05-23 15:12:00'
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''
get_clickhouse_df(q)

,SearchEngineID,Referer,TraficSourceID,UTCStartTime
0,6,http://go.mail.ru/redir?q=Салат %22Уплетайка%22&redir=http:%2F%2Fi.ovkuse.ru%2Fblogs%2Fkulinarija%2Fsalat-upletayka-82935.html&sig=c29a99c4eefeb911902228d8f9e275d8&via_page=1&frm=recomain,2,2017-05-23 15:16:37
1,6,http://go.mail.ru/redir?via_page=1&type=sr&redir=eJzLKCkpKLbS1y9KLC7Qq0zMS0mt0Csq1S8uSSzJzM_TtzQzMjawMLAvqSxItS1OzkhNKc1JZWAwNLE0NTOyMDQyZ9C-6hmhluDsX7c333p5n8pMAHKEG0I,2,2017-05-23 15:16:39
2,6,http://go.mail.ru/redir?via_page=1&type=sr&redir=eJzLKCkpsNLXz8gvSc3RLcssSU0qztZLqtRnYDA0sTQ1M7IwNDdlcK6LqCk7zXt15-7D2eGb2DMB3_oStQ,2,2017-05-23 15:16:43
3,6,http://go.mail.ru/redir?via_page=1&type=sr&redir=eJzLKCkpsNLXT88vqkopSizTKy5I0isq1S_OTM8rLdBPK0pN1WdgMDSxNDUzsjAwNWLQ2C7xdkXFJK72pkXH5k6Y9gMA2dwX6w,2,2017-05-23 15:15:01
4,6,http://go.mail.ru/redir?via_page=1&type=sr&redir=eJzLKCkpsNLXT88vqkopSizTKy5I0isq1S_OTM8rLdBPK0pN1WdgMDSxNDUzsjAwNWLQ2C7xdkXFJK72pkXH5k6Y9gMA2dwX6w,2,2017-05-23 15:15:01


In [113]:
q = '''
    SELECT Referer as url, sum(Sign) as visits FROM visits_all
    WHERE StartDate = today() AND SearchEngineID = 6 AND UTCStartTime >= '2017-05-23 14:35:00'
    GROUP BY url
    ORDER BY visits DESC
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,url,visits
0,http://go.mail.ru/,646828
1,https://go.mail.ru/,80579
2,http://go.mail.ru,40994
3,https://go.mail.ru,1047
4,"http://go.mail.ru/search?fr=main&q=ЗАРАБОТОК,доход,деньги,бесплатно",174
5,http://go.mail.ru/redir?q=Мелания Трамп не подала руки мужу и поставила его в неудобное положение&redir=http:%2F%2Fwww.spletnik.ru%2Fbuzz%2Flove%2F75594-melaniya-tramp-ne-podala-ruki-muzhu.html&sig=b84b965d7e31aa647b6bd63b3bd6f0a4&via_page=1&frm=recomain,37
6,"http://go.mail.ru/search?fr=main&q=Eco-script.RU - сёрфинг раскрутка , заработак , без вложений , без ограничений !",35
7,"http://go.mail.ru/search?fr=main&q=.каталог ножевых изделий, ножи+из дамасской стали, туристические ножи, дамасской стали, дамасская сталь, дамасская сталь ножи, ножи+для кухни, ножи+видео, ножи туристические, ножи+углеродистая сталь, дамасской стали, дамасская сталь ножи, нож дамасской стали купить, ножи+складные, ножи +кухонные, выкидные ножи, ножи+дамасская сталь, ножи+для кухни, туристические+ножи, ножи+углероди",32
8,http://go.mail.ru/search?q=Земля плодородная с доставкой в новосибирске,31
9,http://go.mail.ru/search?q=продажа песка,31
